In [1]:
import os
import dotenv
import cloudinary
import cloudinary.uploader
import pandas as pd
from pymongo import MongoClient
# Load environment variables
dotenv.load_dotenv()

True

In [2]:
# MongoDB Setup
mongo_uri = os.getenv("MONGO_URI")  # From .env
client = MongoClient(mongo_uri)
db = client["coffeeapp"]  # Database Name
products_collection = db["products"]  # Collection Name

# Cloudinary setup

In [3]:

cloudinary.config(
  cloud_name=os.getenv("CLOUDINARY_CLOUD_NAME"),
  api_key=os.getenv("CLOUDINARY_API_KEY"),
  api_secret=os.getenv("CLOUDINARY_API_SECRET")
)



# Upload Data

In [4]:
image_folder_path = './products/images/'

In [5]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [ ]:

def upload_image_to_cloudinary(image_path):
    """Uploads an image to Cloudinary and returns the image URL."""
    try:
        response = cloudinary.uploader.upload(image_path)
        return response["secure_url"]
    except Exception as e:
        print(f"Error uploading {image_path}: {e}")
        return None

In [11]:
# Process and upload data
for index, row in df.iterrows():
    print(f"Processing: {row['name']}")

    image_path = os.path.join(image_folder_path, row["image_path"])
    
    # Upload image to Cloudinary
    image_url = upload_image_to_cloudinary(image_path)
    
    if image_url:
        product_data = row.to_dict()
        product_data.pop("image_path")  # Remove local image path field
        product_data["image_url"] = image_url

        # Insert into MongoDB inside `coffeeapp` project
        products_collection.insert_one(product_data)

print("✅ Data successfully uploaded to MongoDB (coffeeapp) and Cloudinary!")
    

Processing: Cappuccino
Processing: Jumbo Savory Scone
Processing: Latte
Processing: Chocolate Chip Biscotti
Processing: Espresso shot
Processing: Hazelnut Biscotti
Processing: Chocolate Croissant
Processing: Dark chocolate
Processing: Cranberry Scone
Processing: Croissant
Processing: Almond Croissant
Processing: Ginger Biscotti
Processing: Oatmeal Scone
Processing: Ginger Scone
Processing: Chocolate syrup
Processing: Hazelnut syrup
Processing: Carmel syrup
Processing: Sugar Free Vanilla syrup
✅ Data successfully uploaded to MongoDB (coffeeapp) and Cloudinary!


In [11]:
products = list(products_collection.find())
print(products)

[{'_id': ObjectId('67a5b34dc76df69b9ce5b38c'), 'name': 'Cappuccino', 'category': 'Coffee', 'description': 'A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations.', 'ingredients': ['Espresso', 'Steamed Milk', 'Milk Foam'], 'price': 4.5, 'rating': 4.7, 'image_url': 'https://res.cloudinary.com/df6draitv/image/upload/v1738912593/sj2r5ug9hsn0s3tyutzq.jpg'}, {'_id': ObjectId('67a5b350c76df69b9ce5b38d'), 'name': 'Jumbo Savory Scone', 'category': 'Bakery', 'description': 'Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea.', 'ingredients': ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'], 'price': 3.25, '

In [ ]:
products = list(products_collection.find())  # Fetch all products
for product in products:
    product["_id"] = str(product["_id"])
print({"products": products})

{'products': [{'_id': '67a5b34dc76df69b9ce5b38c', 'name': 'Cappuccino', 'category': 'Coffee', 'description': 'A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations.', 'ingredients': ['Espresso', 'Steamed Milk', 'Milk Foam'], 'price': 4.5, 'rating': 4.7, 'image_url': 'https://res.cloudinary.com/df6draitv/image/upload/v1738912593/sj2r5ug9hsn0s3tyutzq.jpg'}, {'_id': '67a5b350c76df69b9ce5b38d', 'name': 'Jumbo Savory Scone', 'category': 'Bakery', 'description': 'Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea.', 'ingredients': ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'], 'price': 3.25, 'rating'